<a href="https://colab.research.google.com/github/preferencial/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
from io import StringIO
from google.colab import drive

# --- Configurações e Constantes ---
DEFAULT_PALETTE = "viridis"
FIGSIZE = (12, 8)

# Estilo dos gráficos
sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 20,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
})

# --- Funções Auxiliares ---
def mount_google_drive():
    """Monta o Google Drive no ambiente Colab."""
    drive.mount('/content/drive')

def get_drive_path(relative_path):
    """Retorna o caminho completo no Google Drive."""
    return os.path.join('/content/drive/MyDrive', relative_path)

def ensure_directory_exists_on_drive(relative_path):
    """Garante que o diretório exista no Google Drive."""
    drive_path = get_drive_path(relative_path)
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
        print(f"Diretório criado no Google Drive: {drive_path}")
    return drive_path

def save_fig(fig, filename, drive_folder_path):
    """Salva o gráfico no Google Drive."""
    try:
        filepath = os.path.join(drive_folder_path, filename)
        fig.savefig(filepath, dpi=300, bbox_inches='tight')
        print(f"Gráfico salvo no Google Drive: {filepath}")
    except Exception as e:
        print(f"Erro ao salvar o gráfico '{filename}': {e}")
    finally:
        plt.close(fig)

def create_figure():
    """Cria uma nova figura com o tamanho padrão."""
    return plt.figure(figsize=FIGSIZE)

# --- Funções de Visualização (Refatoradas) ---
def plot_categorical_count(df, x_col, title, filename, drive_folder_path, hue=None):
    """Plota a contagem de categorias em uma coluna, opcionalmente com hue."""
    try:
        if x_col not in df.columns:
            print(f"Coluna '{x_col}' não encontrada.")
            return
        fig = create_figure()
        sns.countplot(x=x_col, data=df, palette=DEFAULT_PALETTE, hue=hue)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.xticks(rotation=45, ha='right')
        if hue:
            plt.legend(title=hue.replace('_', ' '), loc='upper right')
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar gráfico de contagem: {e}")

def plot_categorical_bar(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Plota um gráfico de barras para variáveis categóricas, opcionalmente com hue."""
    try:
        if x_col not in df.columns or y_col not in df.columns:
            print(f"Colunas '{x_col}' ou '{y_col}' não encontradas.")
            return
        fig = create_figure()
        grouped_data = df.groupby(x_col)[y_col].count().reset_index()  # Count, not sum
        sns.barplot(x=x_col, y=y_col, data=grouped_data, palette=DEFAULT_PALETTE, hue=hue)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        plt.xticks(rotation=45, ha='right')
        if hue:
            plt.legend(title=hue.replace('_', ' '), loc='upper right')
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar gráfico de barras: {e}")


def plot_stacked_bar(df, x_col, hue_col, title, filename, drive_folder_path):
    """Plota um gráfico de barras empilhadas."""
    try:
        if x_col not in df.columns or hue_col not in df.columns:
            print(f"Colunas '{x_col}' ou '{hue_col}' não encontradas.")
            return

        fig = create_figure()
        # Cria uma tabela de contingência (crosstab)
        contingency_table = pd.crosstab(df[x_col], df[hue_col])
        contingency_table.plot(kind='bar', stacked=True, figsize=FIGSIZE, colormap=DEFAULT_PALETTE, ax=plt.gca())

        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel("Contagem")
        plt.xticks(rotation=45, ha='right')
        plt.legend(title=hue_col.replace('_', ' '))
        save_fig(fig, filename, drive_folder_path)

    except Exception as e:
        print(f"Erro ao gerar gráfico de barras empilhadas: {e}")


def plot_grouped_bar(df, x_col, y_col, hue_col, title, filename, drive_folder_path):
    """Plota um gráfico de barras agrupadas (lado a lado)."""
    try:
        if not all(col in df.columns for col in [x_col, y_col, hue_col]):
            print(f"Colunas necessárias não encontradas.")
            return

        fig = create_figure()
        sns.barplot(x=x_col, y=y_col, hue=hue_col, data=df, palette=DEFAULT_PALETTE, errorbar=None) #Removido estimator, pois estamos interessados na contagem
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        plt.xticks(rotation=45, ha='right')
        plt.legend(title=hue_col.replace('_', ' '))
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar gráfico de barras agrupadas: {e}")


def plot_count_with_hue(df, x_col, hue_col, title, filename, drive_folder_path):
    """Plota a contagem de uma variável categórica, diferenciando por outra (hue)."""
    try:
        if x_col not in df.columns or hue_col not in df.columns:
            print(f"Colunas '{x_col}' ou '{hue_col}' não encontradas.")
            return

        fig = create_figure()
        sns.countplot(x=x_col, hue=hue_col, data=df, palette=DEFAULT_PALETTE)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel("Contagem")
        plt.xticks(rotation=45, ha='right')
        plt.legend(title=hue_col.replace('_', ' '))
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar gráfico de contagem com hue: {e}")


def plot_pie_chart(df, column, title, filename, drive_folder_path, limit=10):
    """
    Plota um gráfico de pizza para uma coluna categórica, limitando o número de fatias.
    Agrupa as categorias menos frequentes em "Outros".
    """
    try:
        if column not in df.columns:
            print(f"Coluna '{column}' não encontrada.")
            return

        fig = create_figure()
        # Conta as ocorrências de cada categoria
        counts = df[column].value_counts()

        # Mantém as 'limit' categorias mais frequentes e agrupa o resto em "Outros"
        top_categories = counts.head(limit)
        other_count = counts.iloc[limit:].sum()
        if other_count > 0:  # Só adiciona "Outros" se houver categorias agrupadas
           top_categories['Outros'] = other_count

        top_categories.plot(kind='pie', autopct='%1.1f%%', startangle=140,
                            colors=sns.color_palette(DEFAULT_PALETTE, len(top_categories)),
                            wedgeprops={'edgecolor': 'black'})

        plt.title(title)
        plt.ylabel('')  # Remove o rótulo do eixo y (que é o nome da coluna)
        save_fig(fig, filename, drive_folder_path)

    except Exception as e:
        print(f"Erro ao gerar gráfico de pizza: {e}")


def plot_multiple_pie_charts(df, columns, drive_folder_path, limit=7):
    """
    Plota múltiplos gráficos de pizza, um para cada coluna especificada.
    """
    num_plots = len(columns)
    if num_plots == 0:
        print("Nenhuma coluna especificada para gráficos de pizza.")
        return

    try:
        fig, axes = plt.subplots(1, num_plots, figsize=(5 * num_plots, 5))  # Ajuste o figsize conforme necessário
        if num_plots == 1:  # Se houver apenas um gráfico, axes não será uma lista
            axes = [axes]

        for i, column in enumerate(columns):
            if column not in df.columns:
                print(f"Coluna '{column}' não encontrada.")
                continue

            # Conta as ocorrências de cada categoria
            counts = df[column].value_counts()

            # Mantém as 'limit' categorias mais frequentes e agrupa o resto em "Outros"
            top_categories = counts.head(limit)
            other_count = counts.iloc[limit:].sum()
            if other_count > 0:
                top_categories['Outros'] = other_count

            ax = axes[i]
            top_categories.plot(kind='pie', autopct='%1.1f%%', startangle=140,
                                colors=sns.color_palette(DEFAULT_PALETTE, len(top_categories)),
                                wedgeprops={'edgecolor': 'black'}, ax=ax)
            ax.set_title(column.replace('_', ' '), fontsize=14)
            ax.set_ylabel('')

        fig.suptitle("Distribuição de Categorias", fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Ajusta o layout para evitar sobreposição
        save_fig(fig, "multiple_pie_charts.png", drive_folder_path)

    except Exception as e:
        print(f"Erro ao gerar múltiplos gráficos de pizza: {e}")


# --- Função Principal (main) ---
if __name__ == "__main__":
    try:
        mount_google_drive()
        graficos_drive_path = ensure_directory_exists_on_drive('graficos')

        # Carregar os dados
        url = "https://docs.google.com/spreadsheets/d/1bSdIlZ3y0rCbhBtvw77P1JdxTYx6AfFYM-yWvIuP3nk/export?format=csv"
        response = requests.get(url)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text))

        # Pré-processamento
        df.columns = df.columns.str.replace(' ', '_')  # Padroniza nomes das colunas
        df.dropna(axis=1, how='all', inplace=True)  # Remove colunas completamente vazias
        for col in df.select_dtypes(include=['object']):
            df[col] = df[col].str.strip()  # Remove espaços em branco extras

        print(df.columns) # <--- ADICIONAR ESTA LINHA

        # --- Geração dos Gráficos ---
        # 1. Contagem de Relações
        plot_categorical_count(df, 'Relacao', "1. Contagem de Relações", "1_contagem_relacoes.png", graficos_drive_path)

        # 2. Contagem de Métricas de Avaliação
        plot_categorical_count(df, 'Metrica_Avaliacao', "2. Contagem de Métricas de Avaliação", "2_contagem_metricas.png", graficos_drive_path)

        # 3. Contagem de Bibliotecas Python
        plot_categorical_count(df, 'Biblioteca_Python', "3. Contagem de Bibliotecas Python", "3_contagem_bibliotecas.png", graficos_drive_path)

        # 4. Contagem de Resultados Esperados
        plot_categorical_count(df, 'Resultado_Esperado', "4. Contagem de Resultados Esperados", "4_contagem_resultados.png", graficos_drive_path)

        # 5. Gráfico de Barras: Relação vs. Observações
        plot_categorical_bar(df, 'Relacao', 'Observacoes', "5. Relação vs. Observações", "5_relacao_observacoes.png", graficos_drive_path)

        # --- Novos Gráficos ---

        # 6. Gráfico de Barras Empilhadas: Relação vs. Métrica de Avaliação
        plot_stacked_bar(df, 'Relacao', 'Metrica_Avaliacao', "6. Relação vs. Métrica de Avaliação (Empilhado)", "6_relacao_metrica_empilhado.png", graficos_drive_path)

        # 8. Contagem com Hue: Métrica de Avaliação por Relação
        plot_count_with_hue(df, 'Metrica_Avaliacao', 'Relacao', "8. Métrica de Avaliação por Relação", "8_metrica_por_relacao.png", graficos_drive_path)

        # 9. Gráfico de Pizza: Distribuição de Bibliotecas Python (Top 10 + Outros)
        plot_pie_chart(df, 'Biblioteca_Python', "9. Distribuição de Bibliotecas Python", "9_distribuicao_bibliotecas.png", graficos_drive_path)

        # 10. Multiplos Gráficos de Pizza: Relação, Métrica de Avaliação, Resultado Esperado
        plot_multiple_pie_charts(df, ['Relacao', 'Metrica_Avaliacao', 'Resultado_Esperado'], graficos_drive_path)


        print("Análise completa, gráficos gerados!")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar o URL: {e}")
    except Exception as e:
        print(f"Erro geral no processamento: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Diretório criado no Google Drive: /content/drive/MyDrive/graficos
Index(['Conceito_Principal', 'Conceito_Relacionado', 'Relacao',
       'Metrica_Avaliacao', 'Biblioteca_Python', 'Resultado_Esperado',
       'Observacoes'],
      dtype='object')
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/1_contagem_relacoes.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/2_contagem_metricas.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/3_contagem_bibliotecas.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/4_contagem_resultados.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/5_relacao_observacoes.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/6_relacao_metrica_empilhado.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/8_metrica_por_relacao.png